In [1]:
import tensorflow as tf
from bayes_tec.bayes_opt.maximum_likelihood_tec import *
import numpy as np
float_type = tf.float64

def test_solve():
    
    import numpy as np
    from seaborn import jointplot
    import pylab as plt
    plt.style.use('ggplot')
    freqs = np.linspace(120e6,160e6,20)
    tec_conversion = -8.448e9/freqs
    true_tec = np.random.uniform(-0.2,0.2,size=int(1e3))#np.array([0.004]*1000)
    noise_rads = np.random.uniform(0.05,0.8,size=int(1e3))#np.array([0.3]*1000)# a lot of noise on almost flat TEC is hard
    true_phase = true_tec[...,None] * tec_conversion
    phase = true_phase + noise_rads[...,None]*np.random.normal(size=true_phase.shape)

    tec_min, phase_sigma = solve_ml_tec(phase,freqs,batch_size=int(1e3),verbose=True)
    plt.scatter(true_tec,tec_min)
    plt.xlabel("True tec")
    plt.ylabel("Pred tec")
    plt.show()
   
    
    jointplot(true_tec,tec_min,kind='hex')
    plt.show()
    jointplot(true_tec,tec_min,kind='kde',alpha=0.6,marker='+',color='k')
    plt.show()
    
    plt.scatter(noise_rads, phase_sigma)
    plt.xlabel("Pred phase noise")
    plt.ylabel("True phase noise")
    plt.show()
    jointplot(noise_rads, phase_sigma,kind='hex')
    plt.show()
    jointplot(noise_rads, phase_sigma,kind='kde',alpha=0.6,marker='+',color='k')
    plt.show()

def diagnostics():
    
    import numpy as np
    import pylab as plt
    plt.style.use('ggplot')
    freqs = np.linspace(120e6,160e6,20)
    tec_conversion = -8.448e9/freqs
    true_tec = np.random.uniform(-0.3,0.3,size=1000)#np.array([0.004]*1000)
    noise_rads = np.array([0.3]*1000)# a lot of noise on almost flat TEC is hard
    true_phase = true_tec[...,None] * tec_conversion
    phase = true_phase + noise_rads[...,None]*np.random.normal(size=true_phase.shape)
    
    _tec = true_tec[0]
    
    with tf.Session(graph=tf.Graph()) as sess:
        t_pl = tf.placeholder(float_type)
        phase_pl = tf.placeholder(float_type)
        tec_conversion_pl = tf.placeholder(float_type)
        X_init, Y_init = init_population(phase_pl,tec_conversion_pl,N=5)
        Xcur, Ycur = X_init, Y_init
        X_,Y_,aq_,fmean_,fvar_ = [],[],[],[],[]
        for i in range(21):
            res = bayes_opt_iter(phase_pl, tec_conversion_pl, Xcur, Ycur, max_tec=0.4, t = t_pl)
            X_.append(res.X)
            Y_.append(res.Y)
            aq_.append(res.aq)
            fmean_.append(res.fmean)
            fvar_.append(res.fvar)
            Xcur = res.X
            Ycur = res.Y
        X, Y, aq, fmean, fvar = sess.run([X_, Y_, aq_, fmean_, fvar_], feed_dict={t_pl:1.,
                                                                         phase_pl:phase,
                                                                        tec_conversion_pl:tec_conversion})
        
        indices = (np.arange(Y[-1].shape[0],dtype=np.int64), np.argmin(Y[-1][:,:,0],axis=1), np.zeros(Y[-1].shape[0], dtype=np.int64))
        tec_min = X[-1][indices]
        plt.scatter(tec_min, true_tec)
        plt.xlabel("pred. tec")
        plt.ylabel("true tec")
        plt.title("Scatter of solutions")
        plt.show()

        plt.hist(indices[1],bins=20)
        plt.title("Where was fmin attained")
        plt.xlabel("iteration including random init pop")
        plt.show()

        scatter = []
        for j in range(Y[-1].shape[1]):
            indices = (np.arange(Y[-1].shape[0],dtype=np.int64), np.argmin(Y[-1][:,:j+1,0],axis=1), np.zeros(Y[-1].shape[0], dtype=np.int64))
            tec_j = X[-1][indices]
            scatter.append(np.percentile(np.abs(tec_j - true_tec),95))

        plt.plot(scatter)
        plt.title("95% conf interval of |true_tec - pred_tec|")
        plt.xlabel("iteration")
        plt.ylabel("mean delta tec")
        plt.show()


        tec_array = np.linspace(-0.4, 0.4, 100)
        for i, (x, y, a, f, v) in enumerate(zip(X, Y, aq, fmean, fvar)):
            y = y - y.mean(1,keepdims=True)
            y = y / (np.std(y,axis=1,keepdims=True) + 1e-6)
            
            
            plt.plot(tec_array, f[0,:], label=r'$\mathbb{E}[f]$')
            plt.fill_between(tec_array, f[0,:] - 2*np.sqrt(v[0,:]), f[0,:] + 2*np.sqrt(v[0,:]),alpha=0.5, label=r'$\pm 2\sigma_f$')
            a = a - np.min(a,axis=1,keepdims=True)
            a = 3*a/np.max(a,axis=1,keepdims=True)
            plt.plot(tec_array,a[0,:],label='norm. acquisition func.')
            plt.scatter(x[0, :-1, 0], y[0,:-1, 0],c='k',label='sampled points')
            plt.scatter(x[0, -1, 0], y[0,-1, 0],c='red',label='New sample point')
            plt.vlines(_tec,-2,2,label='global. min',linestyles='--')
            plt.xlabel("tec")
            plt.ylabel("normalized neg-log-likelihood")
            plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
               ncol=2, mode="expand", borderaxespad=0.)
            plt.title("Iteration {}".format(i))
            plt.show()
            
def test_speed(N=1e6):
    
    import numpy as np
    from timeit import default_timer
    freqs = np.linspace(120e6,160e6,20)
    tec_conversion = -8.448e9/freqs
    true_tec = np.random.uniform(-0.2,0.2,size=int(N))#np.array([0.004]*1000)
    noise_rads = np.random.uniform(0.05,0.8,size=int(N))#np.array([0.3]*1000)# a lot of noise on almost flat TEC is hard
    true_phase = true_tec[...,None] * tec_conversion
    phase = true_phase + noise_rads[...,None]*np.random.normal(size=true_phase.shape)

    t0 = default_timer()
    tec_min, phase_sigma = solve_ml_tec(phase,freqs,batch_size=int(N),verbose=True)
    t1 = default_timer()
    t = t1 - t0
    
    print("Time {} [time] {} [samples/s] {} [ms/sample]".format(t,N/t, t/N*1000))
    
            

/net/lofar1/data1/albert/miniconda3/envs/kerastf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# test_speed(N=5e6)

2018-09-15 15:11:39,087 Starting batch 0
2018-09-15 16:20:39,947 Finished batch 0
Time 4150.763997003436 [time] 1204.597515929514 [samples/s] 0.8301527994006873 [ms/sample]


In [ ]:
from bayes_tec.datapack import DataPack
from timeit import default_timer

with DataPack('../../scripts/data/killms_datapack.hdf5') as datapack:
    phase,axes = datapack.phase
    _, freqs = datapack.get_freqs(axes['freq'])
    Npol, Nd, Na, Nf, Nt = phase.shape
    phase = phase.transpose((0,1,2,4,3))
    phase = phase.reshape((-1, Nf))
    t0 = default_timer()
    tec_ml, sigma_ml = solve_ml_tec(phase, freqs, batch_size=int(1e6),max_tec=0.3, n_iter=21, t=1.,num_proposal=75, verbose=True)
    t1 = default_timer()
    print(t1-t0)
    tec_ml = tec_ml.reshape((Npol, Nd, Na, Nt))
    sigma_ml = sigma_ml.reshape((Npol, Nd, Na, Nt))
    with h5py.File('ml_results.hdf5') as f:
        f['tec'] = tec_ml
        f['sigma'] = sigma_ml